<a href="https://colab.research.google.com/github/Develtomas/Python_course/blob/main/mt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
df = spark.read.option("inferSchema", "true").option("header", "true").csv("hdfs:///user/spark/covid/cov.csv")
df.printSchema()

### Task1


In [ ]:
from pyspark.sql.functions import *

df.select("iso_code", "location", (col("total_cases_per_million")/10000).alias("%")).where(col("date")=="2021-03-31").sort(col("%").desc()).show(15)

```
+--------+-------------+------------------+
|iso_code|     location|                 %|
+--------+-------------+------------------+
|     AND|      Andorra|        15.5439073|
|     MNE|   Montenegro|14.523725399999998|
|     CZE|      Czechia|        14.3088484|
|     SMR|   San Marino|        13.9371796|
|     SVN|     Slovenia|10.370805800000001|
|     LUX|   Luxembourg|         9.8473424|
|     ISR|       Israel|          9.625106|
|     USA|United States|          9.203011|
|     SRB|       Serbia|         8.8263286|
|     BHR|      Bahrain|         8.4888601|
|     PAN|       Panama|         8.2287391|
|     PRT|     Portugal|         8.0586997|
|     EST|      Estonia|         8.0226816|
|     SWE|       Sweden|         7.9697443|
|     LTU|    Lithuania|         7.9388647|
+--------+-------------+------------------+
only showing top 15 rows
```

###Task2


In [ ]:
df.where((to_date(col("date")) >= "2021-03-29") & (to_date(col("date")) <= "2021-03-31"))\
    .groupBy("location")\
    .agg(sum("new_cases").alias("cases"))\
    .sort(col("cases").desc())\
    .withColumn("week",lit("march'21 last week")).show(17)

```
+--------------+---------+------------------+
|      location|    cases|              week|
+--------------+---------+------------------+
|         World|1709884.0|march'21 last week|
|        Europe| 600087.0|march'21 last week|
|          Asia| 497396.0|march'21 last week|
|European Union| 479763.0|march'21 last week|
| South America| 340309.0|march'21 last week|
| North America| 239009.0|march'21 last week|
|        Brazil| 214059.0|march'21 last week|
| United States| 197717.0|march'21 last week|
|         India| 182021.0|march'21 last week|
|        Turkey| 109009.0|march'21 last week|
|        France|  98883.0|march'21 last week|
|        Poland|  70726.0|march'21 last week|
|       Germany|  58992.0|march'21 last week|
|         Italy|  52842.0|march'21 last week|
|     Argentina|  40224.0|march'21 last week|
|        Africa|  32221.0|march'21 last week|
|       Ukraine|  30461.0|march'21 last week|
+--------------+---------+------------------+
only showing top 17 rows
```



### Task3



In [ ]:
from pyspark.sql.window import Window

df.where((to_date(col("date")) >= "2021-03-28") & (to_date(col("date")) <= "2021-03-31") & (col("location") == "Russia"))\
    .withColumn("yesterday_cases", lag("new_cases").over(Window.orderBy("date")))\
    .select("date", "yesterday_cases", "new_cases", (col("new_cases")-col("yesterday_cases")).alias("delta"))\
    .show()

```
+-------------------+---------------+---------+------+
|               date|yesterday_cases|new_cases| delta|
+-------------------+---------------+---------+------+
|2021-03-28 00:00:00|           null|   8979.0|  null|
|2021-03-29 00:00:00|         8979.0|   8589.0|-390.0|
|2021-03-30 00:00:00|         8589.0|   8162.0|-427.0|
|2021-03-31 00:00:00|         8162.0|   8156.0|  -6.0|
+-------------------+---------------+---------+------+
```